<a href="https://colab.research.google.com/github/Felipanjos/a3_ia_2022.2/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [307]:
import pandas as pd
import numpy as np
import random
import string
import sklearn
from itertools import groupby

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt

In [272]:
df = pd.read_csv('chip_dataset.csv', index_col=[0])
df.drop('FP16 GFLOPS', axis=1, inplace=True)
df.drop('FP32 GFLOPS', axis=1, inplace=True)
df.drop('FP64 GFLOPS', axis=1, inplace=True)
vendors = list(np.unique(df['Vendor']))

In [273]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [274]:
num_attribs = list(df.describe().columns)
cat_attribs = ['Vendor']

### B. Divisão 

##### a) Amostragem estratificada em Conjunto de Treinamento e Conjunto de Teste

In [275]:
ranges = [0, df['Transistors (million)'].describe()['25%'], 500, 1000, 2000, 6000, 15000, 30000, np.inf]
tiers = ['F', 'E', 'D', 'C', 'B', 'A', 'S', 'S+']

df['Rank'] = pd.cut(df['Transistors (million)'], bins = ranges, labels = tiers)

In [276]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in splitter.split(df, df['Rank']):
  strat_train_set = df.loc[train_index]
  strat_test_set = df.loc[test_index]

In [277]:
def amostragem_estratificada(test_set):
    return (test_set['Rank'].value_counts() / len(test_set))*100    

In [278]:
amostragem_estratificada(strat_test_set)

F     24.963504
C     19.708029
B     17.226277
E     15.328467
D     14.598540
A      6.131387
S      1.751825
S+     0.291971
Name: Rank, dtype: float64

In [ ]:
strat_train_set.drop(columns=['Rank'], inplace=True)
strat_test_set.drop(columns=['Rank'], inplace=True)

### C. Exploração 

##### a) Visualização dos Dados com gráficos de dispersão e histograma

In [279]:
df['Release Date'] = pd.to_datetime(df['Release Date'])

In [ ]:
# ampliar escala 
strat_test_set['Transistors (million)'].hist()

In [ ]:
strat_train_set['Transistors (million)'].hist()

In [ ]:
df.plot(kind='scatter', x='Release Date', y='Transistors (million)')

##### b. Levantamento de hipóteses sobre as distribuições dos dados


##### c. Busca de valueelações (Coeficientes de valueelação)

In [286]:
df['Release Date'] = pd.to_numeric(df['Release Date'])

In [288]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3422 entries, 0 to 3421
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Product                3422 non-null   object  
 1   Type                   3422 non-null   object  
 2   Release Date           3422 non-null   int64   
 3   Process Size (nm)      3422 non-null   float64 
 4   TDP (W)                3422 non-null   float64 
 5   Die Size (mm^2)        3422 non-null   float64 
 6   Transistors (million)  3422 non-null   float64 
 7   Freq (MHz)             3422 non-null   float64 
 8   Foundry                3422 non-null   object  
 9   Vendor                 3422 non-null   object  
 10  Rank                   3422 non-null   category
dtypes: category(1), float64(5), int64(1), object(4)
memory usage: 271.2+ KB


##### Basearemos a análise das valueelações na coluna Transistors (million), que é o nosso objeto de predição

In [326]:
num_attribs

['Process Size (nm)',
 'TDP (W)',
 'Die Size (mm^2)',
 'Transistors (million)',
 'Freq (MHz)']

In [335]:
for value in df.corr()['Transistors (million)']:
    print(value)

-0.002568951085001034
-0.3977453283926615
0.46970179480450375
0.6402633611101106
1.0
-0.04411544348939104


C:\Users\anjos\AppData\Local\Temp\ipykernel_10336\3423880030.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  for value in df.corr()['Transistors (million)']:


In [346]:
colunas_para_calculo = ['Release Date'] + num_attribs
print('Coeficiente de correlação de Pearson entre a coluna Transistors (million) e:')

coor_dict = {}
r_list = []

for value in df.corr()['Transistors (million)']:
    if value == 1:
        r = 'relação linear perfeita e positiva'
    elif value == 0:
        r = 'inexistência linear'
    elif value == -1:
        r = 'relação linear perfeita e negativa'
    elif value > 0:
        r = 'relação linear positiva'
    elif value < 0:
        r = 'relação linear negativa'

    r_list.append(r)

print(df.corr(method='pearson', min_periods=1)['Transistors (million)'], '\n')

count = 0
for col in colunas_para_calculo: 
    coor_dict[col] = r_list[count]
    print(f'{col}: {coor_dict[col]}')
    count += 1


Coeficiente de correlação de Pearson entre a coluna Transistors (million) e:
Release Date            -0.002569
Process Size (nm)       -0.397745
TDP (W)                  0.469702
Die Size (mm^2)          0.640263
Transistors (million)    1.000000
Freq (MHz)              -0.044115
Name: Transistors (million), dtype: float64 

Release Date: relação linear negativa
Process Size (nm): relação linear negativa
TDP (W): relação linear positiva
Die Size (mm^2): relação linear positiva
Transistors (million): relação linear perfeita e positiva
Freq (MHz): relação linear negativa


C:\Users\anjos\AppData\Local\Temp\ipykernel_10336\473720730.py:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  for value in df.corr()['Transistors (million)']:
C:\Users\anjos\AppData\Local\Temp\ipykernel_10336\473720730.py:21: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.corr(method='pearson', min_periods=1)['Transistors (million)'], '\n')


### D. Preparação

##### a) Selecionando 10% das colunas

No dataset utilizado existem somente 9 colunas, dessa forma:

*    10% de 9 = 0.9
*    Aproximando para 1, fica somente uma coluna a ser selecionada como referência para a modificação dos dados



In [ ]:
qtd_colunas = len(df.columns)
p_colunas = round(qtd_colunas * 10 / 100)
coluna_aleatoria = df.sample(n=p_colunas, axis='columns').keys()[0]
colunas_numericas = df.describe().columns
colunas_categoricas = df.describe(exclude=np.number).columns
coluna_aleatoria # escolhendo uma coluna aleatoriamente

##### a) Selecionando 3% dos dados

In [ ]:
p_3 = int(df.shape[0] * 3 / 100)
p_3 

##### a) Excluindo 3% dos dados aleatoriamente

In [ ]:
reg_nulos = df[df[coluna_aleatoria].isnull()]
qtd_reg_nulos = reg_nulos.shape[0]
reg_excluidos = df.sample(n=p_3)
reg_exc_index = reg_excluidos.index.array
df.loc[reg_exc_index, coluna_aleatoria] = np.nan

##### a) Alterando 3% dos dados aleatoriamente

In [ ]:
def gerar_string_aleatoria():
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(10)) 

def gerar_numero_aleatorio():
  return random.randint(0, 999)

In [ ]:
df_selecao_nao_nulo = df[~df[coluna_aleatoria].isnull()]
reg_alterados = df_selecao_nao_nulo.sample(n=p_3)
reg_alter_index = reg_alterados.index.array

In [ ]:
if coluna_aleatoria in colunas_numericas:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_numero_aleatorio()
else:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_string_aleatoria()

#### b) Limpeza dos dados

i. Codificação One-Hot

In [ ]:
encoder = OneHotEncoder()
onehot_encoded_df = pd.DataFrame(encoder.fit_transform(df[['Vendor']]).toarray())
onehot_encoded_df.columns = vendors

onehot_encoded_df = df.join(onehot_encoded_df)
onehot_encoded_df.drop('Vendor', axis=1, inplace=True)
onehot_encoded_df

ii. Estratégia para dados numéricos ausentes - média

A atribuição da mediana geral foi utilizada para lidar com os valores nulos das colunas em que existiam, pelos seguintes motivos:

*   Número pequeno de colunas, logo a remoção completa de cada coluna em que houvesse valor nulo acarretaria na perda desnecessária de informações
*   Facilidade na implementação em comparação com o treinamento de predição e atribuição
*   Coesão com o dataset, visto que logo no primeiro quartil a coluna 'Meta_score' apresenta valor próximo à média


iii. Escalonamento de características: Padronização

iv. Pipeline transformadora

In [ ]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="mean")),        
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ('num', num_pipeline, num_attribs),
        ('cat', OneHotEncoder(), cat_attribs),
    ])

In [ ]:
df_prepared = full_pipeline.fit_transform(df)